In [1]:
pip install brainflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import mne
import BIG
import numpy as np
import time

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations


In [2]:


# BoardShim.enable_dev_board_logger()

# # parser = argparse.ArgumentParser()
# # # use docs to check which parameters are required for specific board, e.g. for Cyton - set serial port
# # parser.add_argument('--timeout', type=int, help='timeout for device discovery or connection', required=False,
# #                     default=0)
# # parser.add_argument('--ip-port', type=int, help='ip port', required=False, default=0)
# # parser.add_argument('--ip-protocol', type=int, help='ip protocol, check IpProtocolType enum', required=False,
# #                     default=0)
# # parser.add_argument('--ip-address', type=str, help='ip address', required=False, default='')
# # parser.add_argument('--serial-port', type=str, help='serial port', required=False, default='')
# # parser.add_argument('--mac-address', type=str, help='mac address', required=False, default='')
# # parser.add_argument('--other-info', type=str, help='other info', required=False, default='')
# # parser.add_argument('--streamer-params', type=str, help='streamer params', required=False, default='')
# # parser.add_argument('--serial-number', type=str, help='serial number', required=False, default='')
# # parser.add_argument('--board-id', type=int, help='board id, check docs to get a list of supported boards',
# #                     required=True)
# # parser.add_argument('--file', type=str, help='file', required=False, default='')
# # args = parser.parse_args()

# params = BrainFlowInputParams()
# params.ip_port = args.ip_port
# params.serial_port = args.serial_port
# params.mac_address = args.mac_address
# params.other_info = args.other_info
# params.serial_number = args.serial_number
# params.ip_address = args.ip_address
# params.ip_protocol = args.ip_protocol
# params.timeout = args.timeout
# params.file = args.file

# board = BoardShim(args.board_id, params)
# board.prepare_session()
# # board.start_stream () # use this for default options
# board.start_stream(45000, args.streamer_params)
# time.sleep(10)
# # data = board.get_current_board_data (256) # get latest 256 packages or less, doesnt remove them from internal buffer
# data = board.get_board_data()  # get all data and remove it from internal buffer
# board.stop_stream()
# board.release_session()

# print(data)




In [3]:
# self.board_id = 38 #BoardIds.BoardIds.MUSE_2_BOARD.value # or BoardIds.NOTION_2_BOARD.value or BoardIds.NOTION_1_BOARD.value
# #self.serial_number = '18721ec44e87864737f236d0be99f56d'
# self.params = BrainFlowInputParams ()
# self.params.board_id = self.board_id
# #self.params.serial_number = self.serial_number
# BoardShim.enable_dev_board_logger ()
# self.board = BoardShim (self.board_id, self.params)
# self.data = np.array(0)
# self.ses_name = ses_name
# self.path = path
# self.expdur = 0

In [3]:
board_id = BoardIds.MUSE_2_BOARD # or BoardIds.NOTION_2_BOARD.value or BoardIds.NOTION_1_BOARD.value
serial_number = '4107-BPDW-BFAB'
params = BrainFlowInputParams ()
params.board_id = board_id
params.serial_number = serial_number
BoardShim.enable_dev_board_logger()
board = BoardShim (board_id, params)
data = np.array(0)
# self.ses_name = ses_name
# self.path = path
expdur = 0

In [6]:
board.prepare_session()

BrainFlowError: BOARD_NOT_READY_ERROR:7 unable to prepare streaming session

In [3]:
sfreq = BoardShim.get_sampling_rate(board_id)

In [6]:
BoardIds.MUSE_2_BOARD.value

38

In [8]:
pwd

'G:\\My Drive\\Lab\\jupyter_notebooks\\Neural Interfaces\\Muse'

In [ ]:
# start recording

def recstart(self):
    self.board.prepare_session()
    self.expdur = time.time()
    self.board.start_stream()


    os.chdir(self.path)
    os.mkdir(self.ses_name)
    os.chdir(self.path + f'/{self.ses_name}')

    f = open("description.txt","w+")
    f.close()
    self.desc = os.getcwd() + '\\description.txt' # Adding description file

    self.start_time = time.time()

In [ ]:
# stop recording and save
def recstop(self):
    self.data = self.board.get_board_data()
    self.board.stop_stream()
    self.expdur = time.time() - self.expdur
    self.board.release_session()

    self.eeg_channels = BoardShim.get_eeg_channels(BoardIds.BoardIds.MUSE_2_BOARD (38).value)
    self.eeg_data = self.data[self.eeg_channels, :]
    self.eeg_data = self.eeg_data / 1000000  # BrainFlow returns uV, convert to V for MNE

    # Creating MNE objects from brainflow data arrays
    self.ch_types = ['eeg'] * len(self.eeg_channels)
    self.ch_names = BoardShim.get_eeg_names(BoardIds.MUSE_2_BOARD.value)
    self.sfreq = BoardShim.get_sampling_rate(BoardIds.MUSE_2_BOARD.value)
    self.info = mne.create_info(ch_names=self.ch_names, sfreq=self.sfreq, ch_types=self.ch_types)
    self.raw = mne.io.RawArray(self.eeg_data, self.info)



In [ ]:
def show_desc(self):
    with open(self.desc, 'r') as f:
        print(f.read())
        f.close()

In [ ]:
def add_timestamp(self, input_str):
    timestamp = time.time() - self.start_time
    timestamp = round(timestamp, 2)
    with open(self.desc, 'a') as f:
        f.write(f'\n[@ {timestamp} s]: {input_str}')
        f.close()

In [ ]:
def save(obj):
    
    with open(f'{obj.ses_name}.pickle', 'wb') as f:
        pickle.dump(obj, f)

    f.close()

    fig = plt.figure()
    fig = obj.raw.plot_psd(fmax=50)
    fig.savefig('psd', format='jpg')
    

In [ ]:
def load(ses_name):
    print(os.getcwd())
    os.chdir(r'G:\My Drive\Lab\jupyter_notebooks\Neural Interfaces\recordings' + f'/{ses_name}')
    print(os.getcwd())
    with open(f'{ses_name}.pickle', mode='rb') as f:
        obj = pickle.load(f)
    
    return obj